In [1]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import pandas as pd

In [2]:
MODEL_NAME = "IlyaGusev/saiga_mistral_7b"
DEFAULT_MESSAGE_TEMPLATE = "<s>{role}\n{content}</s>"
DEFAULT_RESPONSE_TEMPLATE = "<s>bot\n"
DEFAULT_SYSTEM_PROMPT = "Ты — Сайга, русскоязычный автоматический ассистент. Ты разговариваешь с людьми и помогаешь им."

In [3]:
class Conversation:
    def __init__(
        self,
        message_template=DEFAULT_MESSAGE_TEMPLATE,
        system_prompt=DEFAULT_SYSTEM_PROMPT,
        response_template=DEFAULT_RESPONSE_TEMPLATE
    ):
        self.message_template = message_template
        self.response_template = response_template
        self.messages = [{
            "role": "system",
            "content": system_prompt
        }]

    def add_user_message(self, message):
        self.messages.append({
            "role": "user",
            "content": message
        })

    def add_bot_message(self, message):
        self.messages.append({
            "role": "bot",
            "content": message
        })

    def get_prompt(self, tokenizer):
        final_text = ""
        for message in self.messages:
            message_text = self.message_template.format(**message)
            final_text += message_text
        final_text += DEFAULT_RESPONSE_TEMPLATE
        return final_text.strip()

In [4]:
def generate(model, tokenizer, prompt, generation_config):
    data = tokenizer(prompt, return_tensors="pt", add_special_tokens=False)
    data = {k: v.to(model.device) for k, v in data.items()}
    output_ids = model.generate(
        **data,
        generation_config=generation_config
    )[0]
    output_ids = output_ids[len(data["input_ids"][0]):]
    output = tokenizer.decode(output_ids, skip_special_tokens=True)
    return output.strip()

In [5]:
config = PeftConfig.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    load_in_8bit=True,
    torch_dtype=torch.float16,
    device_map="auto"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
model = PeftModel.from_pretrained(
    model,
    MODEL_NAME,
    torch_dtype=torch.float16
)
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32002, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): Linear8bitLt(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
              )
              (k_proj): Linear8bitLt(
                (lora_dropout): Modul

In [7]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
generation_config = GenerationConfig.from_pretrained(MODEL_NAME)
print(generation_config)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_new_tokens": 1536,
  "no_repeat_ngram_size": 15,
  "pad_token_id": 0,
  "repetition_penalty": 1.1,
  "temperature": 0.2,
  "top_k": 40,
  "top_p": 0.9
}



In [8]:
def model_answer(input):
    conversation = Conversation()
    conversation.add_user_message(input)
    prompt = conversation.get_prompt(tokenizer)
    model_output = generate(model, tokenizer, prompt, generation_config)
    torch.cuda.empty_cache()
    return model_output

In [78]:
def create_prompt(text):
    prompt = f'''Анализируйте следующий текст сообщения пользователя из социальной сети и найдите все сущности, ограничиваясь списком: номера телефонов, фамилии, имена, отчества, адреса, даты, названия организаций. Формируйте результат в виде JSON, включая только те типы сущностей, которые присутствуют в тексте. Если определённого типа сущностей нет в тексте, их не следует включать в итоговый JSON.\n\nТекст сообщения:\n{text}\n\nОтвет:'''
    return prompt

In [30]:
df = pd.read_csv("train_dataset_train_variant2.csv")

In [31]:
texts = df['Текст инцидента'].to_list()

In [32]:
texts

[' Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.Уважаемая Администрация, сделайте с этим что нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо.',
 ' Пермь г, 79194692145. В Перми с ноября 2021 года не работает социальное такси. Каким образом можно получить льготу по проезду в такси в соц учреждения инвалиду 2гр.пррезд в общественном транспорте не возможен. Да и проездного льготного не представляется',
 ' Добрый день . Скажите пожалуйста если подовала на пособие с 3 до 7 2 декабря , когда можно повторно подать ? вроде за 30 дней можно',
 ' Каждая из них не о чем. Люди на остановках хотят укрыться от непогоды или слишком погоды. Присесть, поставить сумку. Лавочки на полторы попы? Отсутствие или намек на присутствие спинки? Конструкторы

In [79]:
for i in range(10):
    text = texts[i]
    model_input = create_prompt(text)
    model_output = model_answer(model_input)
    print(f'{i}=============\nText:\n{text}\nModel:\n{model_output}\n=============')

0=============
Text:
 Добрый день. Сегодня, 20.08.22, моя мать шла по улице Ленина между домами 96 и 94. Фонари не горят, упала в яму, которую не видно. Сильно ударилась, остались синяки, очень больно. Благо шла не одна.Уважаемая Администрация, сделайте с этим что нибудь, да и не только с этим. Ходить опасно не только взрослым, но и детям. Если бы упал маленький ребёнок, было бы намного хуже. Фото прилагаю. Спасибо.
Model:
{
  "names": [
    "Мать"
  ],
  "dates": [
    "20.08.22"
  ],
  "organizations": [
    "Администрация"
  ]
}
1=============
Text:
 Пермь г, 79194692145. В Перми с ноября 2021 года не работает социальное такси. Каким образом можно получить льготу по проезду в такси в соц учреждения инвалиду 2гр.пррезд в общественном транспорте не возможен. Да и проездного льготного не представляется
Model:
{
  "phone_numbers": [
    "79194692145"
  ],
  "cities": [
    "Пермь"
  ],
  "dates": [
    "ноября 2021 года"
  ],
  "organizations": [
    "социальное такси",
    "соц учрежде